In [1]:
from IPython import display
import ee
import geemap
import geemap.colormaps as cm
import pandas as pd
import ee
import datetime
import intake
from pathlib import Path
import numpy as np

In [2]:
ee.Authenticate()
ee.Initialize(project='sentinel-treeclassification')

In [3]:
catalog = intake.open_catalog(Path('catalog.yml'))
source = getattr(catalog, 'treesat')
target = source.metadata['categories']['multi']
gdf = source.read()[source.metadata['usecols']]
# Geopandas does not seem to always recognise the crs.
gdf.crs = 25832
gdf = gdf.to_crs(epsg=4326)
# Add some padding to avoid border polygons being cut off.
bbox = ee.Geometry.BBox(*(gdf.geometry.total_bounds + 0.01))

TypeError: GeoPandasFileSource.__init__() got an unexpected keyword argument 'crs'

In [ ]:
class SentinelGetter:
    def mask_s2_clouds(self, image):
      # Quality assessment with resolution in meters
      qa = image.select('QA60')
      # Bits 10 and 11 are clouds and cirrus, respectively.
      cloud_bit_mask = 1 << 10
      cirrus_bit_mask = 1 << 11
      # Both flags should be set to zero, indicating clear conditions.
      mask = (
          qa.bitwiseAnd(cloud_bit_mask)
          .eq(0)
          .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
      )
      return image.updateMask(mask)

    def get_image(self, start_date, end_date):
        selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']
        image = (
            ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterDate(start_date, end_date)
            # Pre-filter to get less cloudy granules.
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            .map(self.mask_s2_clouds)
            .select(selected_bands)
        )
        return image

rgb_bands = ['B4', 'B3', 'B2']
visualization = {
    'min': 1,
    'max': 3000,
    'bands': rgb_bands
}

month_jump = 3
start_date = datetime.datetime(2023, 6, 1)
end_month = (start_date.month + month_jump)%12 or 12
end_date = start_date.replace(
    month=end_month, 
    year=start_date.year + (start_date.month + month_jump)//13)

image = SentinelGetter().get_image(start_date, end_date)
m = geemap.geemap.Map()

center = (np.array(bbox.getInfo()['coordinates'][0][2]) + np.array(bbox.getInfo()['coordinates'][0][0]))/2

m.set_center(*center, 7)

style = {"stroke": True, "color": "green",
         "weight": 2, "opacity": 1, "fillOpacity": 0.1
}

m.add_gdf(gdf.head(1000), layer_name='treesat', style=style)

m.addLayer(image.median().clip(bbox), visualization, 'RGB')

earth_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
m.add_tile_layer(earth_url, name='Google Earth', attribution='Google')

m.addLayerControl(position='topright')
m